# Test the accuracy of the TFLite model 

This project tests the accuracy of a TensorFlow Lite model for audio classification. The model classifies audio clips into two categories: dog barks and sirens. Mel spectrograms are extracted from audio clips and used as input to the model. The project loads a pretrained quantized TFLite model, runs inference on a test set, and calculates the accuracy compared to the ground truth labels.

In [1]:
import os
import numpy as np
import tensorflow as tf
import librosa
from tensorflow.image import resize
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# Set up data paths and hyperparameters
data_dir = 'data'
classes = ['dog_bark', 'siren']
target_shape = (128, 128)

### Data Loading and Preprocessing

The `load_and_preprocess_data` function loads audio clips from disk, extracts mel spectrograms using librosa, resizes the spectrograms to the expected input shape, and collects the data and corresponding labels. The raw audio and sample rate are loaded, then a mel spectrogram feature representation is extracted. The spectrograms are resized to match what the model expects as input. The data and labels are gathered in arrays.

In [3]:
# Function to load audio data from disk and preprocess into mel spectrograms
def load_and_preprocess_data(data_dir, classes, target_shape):
    # Initialize empty lists to hold data and labels
    data = []
    labels = []

    # Loop through class names
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                # Load audio data and sample rate
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)

                # Extract mel spectrogram
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = mel_spectrogram.squeeze()

                # Resize spectrogram
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)

                # Append data and labels
                data.append(mel_spectrogram)
                labels.append(i)

    return np.array(data), np.array(labels)

In [5]:
# Load and preprocess data
data, labels = load_and_preprocess_data(data_dir, classes, target_shape)

In [6]:
# One-hot encode labels
labels = to_categorical(labels, num_classes=len(classes))

### Train/Test Split

The preprocessed data and labels are split into train and test sets using scikit-learn's `train_test_split`. 20% of the data is held out for testing.

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

### Load and Test TFLite Model

A quantized TFLite model is loaded from disk and the tensors are allocated. The input and output tensor details are fetched. Batches of test data are passed to the interpreter and inference is performed. The predictions are compared to the ground truth labels to calculate the accuracy. Finally, the overall test accuracy is printed.

In [8]:
# Load TFLite model
# interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter = tf.lite.Interpreter(model_path="quant_model.tflite")

In [9]:
# Allocate tensors
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
# Function to calculate accuracy
def compute_accuracy(output, y):
    preds = np.argmax(output, axis=1)
    return np.sum(preds == np.argmax(y, axis=1)) / len(y)

In [13]:
# Test model
batch_size = 32
num_samples = X_test.shape[0]
test_accuracy = 0

for i in range(0, num_samples, batch_size):
    test_data = X_test[i:i + batch_size]

    for j in range(len(test_data)):
        current_sample = test_data[j]
        current_sample = np.expand_dims(current_sample, axis=0)  # Add batch dimension
        interpreter.set_tensor(input_details[0]['index'], current_sample)
        interpreter.invoke()

        output = interpreter.get_tensor(output_details[0]['index'])

        test_accuracy += compute_accuracy(output, y_test[i + j:i + j + 1])

test_accuracy /= num_samples

print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9093264248704663
